# Generating the pairs of data for Contrastive Learning

In [4]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import audiomentations
#from audiomentations import Compose, PitchShift
#import torch_audiomentations
from torch_audiomentations import Compose, PitchShift, TimeInversion
import torchaudio
import numpy as np

In [26]:
AUDIO_DIR = "../valid_data/"
CSV_PATH = "../valid_data/metadata_compiled_valid_edited.csv"

df = pd.read_csv(CSV_PATH, header=0)
#print('df: ', df)
for row in tqdm(df.itertuples()):
    print('line: ', row[2])

    filename = row[2] + '.wav'
    #use torch audiomentations to augment the data and save the files

    signal, sr = torchaudio.load(AUDIO_DIR + filename)

    audio_tranform = Compose(
        transforms=[
            PitchShift(
                mode = "per_example",
                p=0.5,
                sample_rate=sr,
            ), 
            TimeInversion(
                mode = "per_example",
                p=0.5,
         )
        ]
    )
   
   #change signal from [1, num_samples] to [1, 1, num_samples]
    signal = signal.unsqueeze(1)
    print('signal shape: ', signal.shape)

    aug1 = audio_tranform(signal, sr)
    aug2 = audio_tranform(signal, sr)

    

    #change aug1 and aug2 from [1, 1, num_samples] to [1, num_samples]
    aug1 = aug1.squeeze(1)
    aug2 = aug2.squeeze(1)

    print('aug1 shape: ', aug1.shape)
    print('aug2 shape: ', aug2.shape)

    torchaudio.save(AUDIO_DIR + row[2] + '_aug1.wav', aug1, sr)
    torchaudio.save(AUDIO_DIR + row[2] + '_aug2.wav', aug2, sr)

    #break


line:  00039425-7f3a-42aa-ac13-834aaa2b6b92
signal shape:  torch.Size([1, 1, 469440])


/Users/serenazhang/Documents/cs230/coughvid/.venv/lib/python3.10/site-packages/torch_audiomentations/core/transforms_interface.py:77: FutureWarning: Transforms now expect an `output_type` argument that currently defaults to 'tensor', will default to 'dict' in v0.12, and will be removed in v0.13. Make sure to update your code to something like:
  >>> augment = PitchShift(..., output_type='dict')
  >>> augmented_samples = augment(samples).samples
  warnings.warn(
/Users/serenazhang/Documents/cs230/coughvid/.venv/lib/python3.10/site-packages/torch_audiomentations/core/transforms_interface.py:77: FutureWarning: Transforms now expect an `output_type` argument that currently defaults to 'tensor', will default to 'dict' in v0.12, and will be removed in v0.13. Make sure to update your code to something like:
  >>> augment = TimeInversion(..., output_type='dict')
  >>> augmented_samples = augment(samples).samples
  warnings.warn(
/Users/serenazhang/Documents/cs230/coughvid/.venv/lib/python3.10/

aug1 shape:  torch.Size([1, 469440])
aug2 shape:  torch.Size([1, 469440])
